In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import os
from scipy.optimize import curve_fit

## Data preparation

### Load data

In [14]:
chirp_direction = 'chirp_up'
data_file_path = r'C:\Users\vaish\dsvv_atomionics\compiled_data\compiled_' + f'{chirp_direction}.pkl'
data_cols = [
		'chirp_rate', 'fraction',
		'CA+_mean', 'CA+_std_dev', 'CA+_percentile_0', 'CA+_percentile_10', 'CA+_percentile_20', 
		'CA+_percentile_30', 'CA+_percentile_40', 'CA+_percentile_50', 'CA+_percentile_60', 'CA+_percentile_70', 
		'CA+_percentile_80', 'CA+_percentile_90', 'CA+_percentile_100', 'CA-_mean', 'CA-_std_dev', 'CA-_percentile_0', 
		'CA-_percentile_10', 'CA-_percentile_20', 'CA-_percentile_30', 'CA-_percentile_40', 'CA-_percentile_50', 
		'CA-_percentile_60', 'CA-_percentile_70', 'CA-_percentile_80', 'CA-_percentile_90', 'CA-_percentile_100'
	]
data = pd.DataFrame(pd.read_pickle(data_file_path), columns=data_cols)

# 80-20 train/test split
train_data_df = data.sample(frac=0.8, random_state=0)
test_data_df = data.drop(train_data_df.index)

print(train_data_df.shape)
# print(train_data.head())
print(test_data_df.shape)
# print(test_data.head())

(63, 28)
(16, 28)


### Constant parameters

In [ ]:
# I'm not sure of the calculations here, copied from the original code
# constants for evaluation: keff, bigT, chirp_direction
speed_of_light_in_vacuum = 299792458
f1 = 384.2304844685e12 - 72.9113e6 - 2.56300597908911e9 + 61.625e6
f2 = f1 + 6.725e9 + 112.06936e6
k1 = 2*np.pi/(speed_of_light_in_vacuum/f1)
k2 = 2*np.pi/(speed_of_light_in_vacuum/f2)
Keff = k1+k2

bigT = 10e-3

## Custom Loss Function

In [ ]:
def sine(alpha, contrast, acc_due_to_gravity, fringe_offset):
    phi = (Keff * acc_due_to_gravity - 2 * np.pi * alpha) * (bigT**2)
    return (-contrast * np.cos(phi) + fringe_offset)

In [ ]:
def get_popt_for_best_fit(corrected_chirp_rates: np.ndarray, observed_fractions: np.ndarray):
	contrast = (np.max(observed_fractions) - np.min(observed_fractions)) / 2
	fringe_offset = np.mean(observed_fractions)
	g_guess = -9.78306 if chirp_direction == 'chirp_down' else 9.77997
	period = 780e-9/(2*bigT**2)

	sine_function_parameters = [contrast, g_guess, fringe_offset]
	bounds = ([0, g_guess - 0.501*period, -np.inf], [1, g_guess + 0.501*period, np.inf])
	popt, _ = curve_fit(sine, corrected_chirp_rates, observed_fractions, p0=sine_function_parameters, maxfev=1000000, bounds=bounds)
	return popt

In [ ]:
def calculate_mse_loss_with_popt(outputs: torch.TensorType, observed_fractions: torch.TensorType, *popt):
	print(popt)
	tensor0 = torch.tensor([Keff * popt[1] * bigT**2 for i in range(len(outputs))], dtype=torch.float32)
	tensor1 = outputs * bigT**2 * 2 * torch.pi
	phi_tensor = torch.cos(tensor0 - tensor1)
	predicted_fractions_tensor = phi_tensor * (-popt[0]) + popt[2]
	mse_loss = nn.MSELoss()
	loss = mse_loss(predicted_fractions_tensor, observed_fractions)
	return loss

## Benchmark MSE score

In [16]:
test_chirp_rates = test_data_df['chirp_rate'].to_numpy()
test_fractions = test_data_df['fraction'].to_numpy()
popt = get_popt_for_best_fit(test_chirp_rates, test_fractions)

residuals = test_fractions - sine(test_chirp_rates, *popt)
mse = np.mean(residuals**2)
print(f'MSE: {mse}')

MSE: 0.007109255364012863


## Training

### Neural Network

In [ ]:
class TransformationNN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(TransformationNN, self).__init__()
        # Define the layers of the network: 3 fully connected layers 27 -> 64 -> 64 -> 1
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, output_dim)
    
    def forward(self, x):
        x = torch.tanh(self.fc1(x))
        x = torch.tanh(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
training_data_tensor = torch.tensor(train_data_df.drop(columns=['fraction']).values, dtype=torch.float32)
observed_fractions_tensor = torch.tensor(train_data_df['fraction'].values, dtype=torch.float32)
observed_fractions_ndarray = train_data_df['fraction'].to_numpy()

In [ ]:
# Initialize the neural network
model = TransformationNN(input_dim=27, output_dim=1)
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [ ]:

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    
    # Forward pass: Compute predicted y by passing x to the model
    outputs = model(training_data_tensor)
    
    outputs_clone = outputs.clone()
    corrected_chirp_rates_ndarray = outputs_clone.detach().cpu().squeeze().numpy()
    
    popt = get_popt_for_best_fit(corrected_chirp_rates_ndarray, observed_fractions_ndarray)    
    loss = calculate_mse_loss_with_popt(outputs, observed_fractions_tensor, *popt)
    
    # Print the loss
    print(f'Epoch [{epoch + 1}/{num_epochs}], MSE: {loss.item():.4f}')

    # Zero gradients, perform a backward pass, and update the weights
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

(0.9999999987234874, 9.779328702815338, 0.5565219132815651)
Epoch [1/10], MSE: 0.0063
(0.9999992247590119, 9.779327206883245, 0.5533873427543377)
Epoch [2/10], MSE: 0.0063
(0.9999836580306009, 9.779327402711502, 0.5540719384797644)
Epoch [3/10], MSE: 0.0063


c:\Users\vaish\AppData\Local\Programs\Python\Python38\lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([63])) that is different to the input size (torch.Size([63, 63])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


(0.999991717389824, 9.779326938863631, 0.5534702339827707)
Epoch [4/10], MSE: 0.0063
(0.9999995224656045, 9.781277636613908, 0.5536143492630391)
Epoch [5/10], MSE: 0.0063
(0.9999803283969395, 9.781276221171815, 0.5558222759562409)
Epoch [6/10], MSE: 0.0063
(0.9999999787329349, 9.781278999476632, 0.5514831239388188)
Epoch [7/10], MSE: 0.0063
(0.9999663025313382, 9.781274553192866, 0.55853448975553)
Epoch [8/10], MSE: 0.0063
(0.13574413192951723, 9.779794937122182, 0.6464742729484514)
Epoch [9/10], MSE: 0.0063
(0.9999505587606222, 9.779327817238034, 0.5548619233677138)
Epoch [10/10], MSE: 0.0063


### Genetic Algorithm